# Chapter 9: Bridging Finite and Super-population Causal Inference

In [1]:
from joblib import Parallel, delayed

import numpy as np
import statsmodels.api as sm

np.random.seed(42)
%load_ext autoreload
%autoreload 1

%load_ext watermark
%watermark --iversions



scipy      : 1.13.0
statsmodels: 0.14.1
numpy      : 1.26.4



In [2]:
def linestimator(Z, Y, X):
    X = (X - X.mean(axis=0)) / X.std(axis=0)
    n, p = X.shape
    # fully interacted OLS
    Xmat = np.c_[sm.add_constant(Z), X, Z.reshape(-1, 1) * X]
    m = sm.OLS(Y, Xmat).fit(cov_type="HC2")
    est, vehw = m.params[1], m.bse[1] ** 2
    # super-population correction
    inter = m.params[-p:]  # (β_1 - β_0) term - last p elements of coef
    # (β_1 - β_0)' Σ (β_1 - β_0) / n
    superCorr = (inter @ np.cov(X.T) @ inter)/n
    vsuper = vehw + superCorr
    return est, np.sqrt(vehw), np.sqrt(vsuper)

In [3]:
def onerepl(*args):
    n = 500
    X = np.random.normal(0, 1, n * 2).reshape(n, 2)
    Y0 = X[:, 0] + X[:, 0] ** 2 + np.random.uniform(-0.5, 0.5, n)
    Y1 = X[:, 1] + X[:, 1] ** 2 + np.random.uniform(-1, 1, n)
    Z = np.random.binomial(1, 0.6, n)
    Y = Y0 * (1 - Z) + Y1 * Z
    return linestimator(Z, Y, X)
onerepl()

(0.052230404017171474, 0.1475302340448403, 0.1633386978782156)

In [4]:
nrep, k = 2_000, 8
results = Parallel(n_jobs=k)(delayed(onerepl)(i) for i in range(nrep))
simres = np.vstack(results)

In [5]:
# bias, estimated EHW SE, estimated super-population SE
simres[:, 0].mean(), simres[:, 1].mean(), simres[:, 2].mean()

(0.0033900784280582142, 0.13559452100782549, 0.15029308662381266)

In [6]:
# empirical SD
simres[:, 0].std()

0.14731850757623555

In [7]:
# EHW coverage
np.mean(
    (simres[:, 0] - 1.96 * simres[:, 1]) * (simres[:, 0] + 1.96 * simres[:, 1]) <= 0
)

0.926

EHW has below nominal coverage for superpopulation.

In [8]:
# superpop coverage
np.mean(
    (simres[:, 0] - 1.96 * simres[:, 2]) * (simres[:, 0] + 1.96 * simres[:, 2]) <= 0
)

0.9515

Superpopn is above nom coverage for superpopulation.